In [ ]:
import os
import json
from datetime import date, datetime, timedelta
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import pandas as pd
from libsstock import (
    loadStocks, getYFdate, getInfoBuy, computeIchimoku, fillNameFromYF,
    graphIchimoku, graphBestGain, graphWorseGain, graphCashLock, graphRendement
)
from stockList import allStocks, stockList
from plotly_tools import genIMGfromFile
from IPython.display import Image as ImageD

In [ ]:
df = loadStocks('mystocks.json')
display(df)

In [ ]:
#display(ImageD(data=genIMGfromFile(graphBestGain(df), 'img.png', scale=1.3, width=500, height=500)))
#display(ImageD(data=genIMGfromFile(graphWorseGain(df), 'img.png', scale=1.3, width=500, height=500)))
#display(ImageD(data=genIMGfromFile(graphCashLock(df), 'img.png', scale=1.3, width=500, height=500)))
display(ImageD(data=genIMGfromFile(graphRendement(df), 'img.png', scale=1.3, width=800, height=500)))

In [ ]:
historyData = getYFdate(df, startDate=date.today() - timedelta(days=365+41), stopDate=date.today() + timedelta(days=1))
listStockNames = [df.loc[ind]['stockname'] for ind in df.index]
for ind in range(len(listStockNames)):
    histo = computeIchimoku(historyData[ind])
    display(ImageD(data=genIMGfromFile(graphIchimoku(df[df['stockname'] == listStockNames[ind]], histo), 'img.png', scale=1.3)))

In [ ]:
# fill name in empty fields
jsonToUpdate = 'stockprospects.json'
df = pd.read_json(jsonToUpdate, orient='index')
df = fillNameFromYF(df)
df.to_json(jsonToUpdate, orient='index')
display(df)
historyData = getYFdate(df, startDate=date.today() - timedelta(days=365+41), stopDate=date.today() + timedelta(days=1))

In [ ]:
[{'label':df.loc[ind]['name'], 'value':df.loc[ind]['stockname']} for ind in df.index]

In [ ]:
dateStart = datetime.now() - timedelta(days=200)
dateStop = datetime.now()
df = pd.DataFrame([{'stockname': 'STM.PA',},])
dfData = loadStocks('mystocks.json')
dfData = dfData[dfData['stockname'] == df.iloc[0]['stockname']]
if len(dfData) > 0:
    df = dfData
else:
    df = fillNameFromYF(df)
dfHist = getYFdate(df, dateStart, dateStop)
dfHist = computeIchimoku(dfHist[0])
display(ImageD(data=genIMGfromFile(graphIchimoku(df, dfHist).update({'layout': dict(template="plotly_dark", xaxis_rangeslider_visible=False)}), 'img.png', scale=1.3, width=800, height=500)))

In [ ]:
listStockNames = [df.loc[ind]['stockname'] for ind in df.index]
for ind in range(len(listStockNames)):
    histo = computeIchimoku(historyData[ind])
    display(ImageD(data=genIMGfromFile(graphIchimoku(df[df['stockname'] == listStockNames[ind]], histo), 'img.png', scale=1.3)))